**I asked a question on [math.stackexchange.com](math.stackexchange.com), and recieved a great answer. This is my attempt to implement the insighful solution** ([link to question](http://math.stackexchange.com/questions/1350456/probability-a-blackjack-dealer-will-bust-if-you-know-their-score-and-know-the-ex))

**After some thought and reading comments on math stackexchange, i decided to implement the recursive solution rather than the linear algebra one for now**

In [13]:
from __future__ import division

def hit_hand(score, soft, value):
    '''given a hand and a card, returns the new hand
    
    arguments: 
    score: the current score of the hand (int)
    soft: whether or not the score is soft (boolean)
    value: the value of the card to add to the hand
    
    returns (score, soft): the score and whether or not the score is soft of the resulting hand
    '''
    if soft and score + value > 21:
        score = score - 10 + value
        soft = False
    elif value == 1 and score + 11 <= 16:
        score += 11
        soft = True
    else:
        score += value
    return (score, soft)


def p_bust(hand, shoe, p=0.0):
    '''given a hand and a shoe, returns the probability of busting
    assumes dealer stands on soft 17
    
    arguments:
    hand: tuple containing score and whether score is soft (tuple)
    shoe: the number of cards of each value left to draw (list)
    
    keyword_arguments:
    p: aggregator for probability of busting (default=0.0)
    
    returns p: probability of busting (float)
    '''
    score, soft = hand
    if score <= 16:
        for card in range(10):
            score_after_hitting, soft = hit_hand(score, soft, card + 1)
            if score_after_hitting >= 22:
                p += shoe[card]/sum(shoe)
            elif score_after_hitting <= 16:
                new_shoe = [c - 1 if i == card else c for i,c in enumerate(shoe)]
                p += new_shoe[card]/sum(new_shoe) * p_bust((score_after_hitting, soft), new_shoe, p)
    return p


def possible_hands(showing, shoe):
    ''' generates a list of all possible hands and the probability 
    of seeing that hand given a shoe and an upcard
    
    arguments:
    showing: the value of the card that is showing
    shoe: the number of cards of each value left to draw (list)
    
    returns [((score, scoft) p), ...]: list of the hands (tuple of the score and whether it is soft) and probability of hand
    '''
    hands = []
    for card in range(10):
        if shoe[card] != 0:
            p_card = shoe[card] / sum(shoe)
            if showing == 1:
                hands.append((hit_hand(11, True, card + 1), p_card))
            else:
                hands.append((hit_hand(showing, False, card + 1), p_card))
    return hands

master_shoe = [4*6 for _ in range (9)] + [4*4*6]

for card in range(10):
    shoe = [c - 1 if i == card else c for i,c in enumerate(master_shoe)]
    print 'Probability of bust showing %d: %f' % (card + 1, sum([p_bust(hand, shoe)*p_card for hand, p_card in possible_hands(card + 1, shoe)]))


Probability of bust showing 1: 0.193901
Probability of bust showing 2: 0.488359
Probability of bust showing 3: 0.469718
Probability of bust showing 4: 0.464140
Probability of bust showing 5: 0.465058
Probability of bust showing 6: 0.473674
Probability of bust showing 7: 0.300726
Probability of bust showing 8: 0.274188
Probability of bust showing 9: 0.252677
Probability of bust showing 10: 0.231862


----------
<blockquote>
I think this question is related to a broader question that I ask myself all the time in research: what does it mean to have a "nice solution" to a problem? When I was young, I was taught that a "nice solution" is a formula for the thing you want to calculate, but that's not always true! Having a forumla for something often tells you very little about it, and other descriptions are often much more useful from a practical point of view.

I'm not sure whether the description of the bust probability given above is much use, but for this problem, I suspect that a linear algebraic description of this kind will be more useful than a formula.
</blockquote>